In [8]:
import numpy as np
import csv
from collections import defaultdict

train_data = defaultdict(dict)

with open("../data/trainIdx2_matrix.txt", 'r') as fTrain:
    for line in fTrain:
        trainUserID, trainItemID, trainRating = line.strip().split('|')
        train_data[trainUserID][trainItemID] = trainRating

with open("../data/testTrack_hierarchy.txt", 'r') as fTest, open("../data/proccessed/output1.csv", 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(["UserId", "TrackId", "AlbumId", "ArtistId", "TrackRating",
                         "AlbumRating", "ArtistRating", "TotalScore", "Predictor"])

    trackID_vec = [0] * 6
    albumID_vec = [0] * 6
    artistID_vec = [0] * 6
    lastUserID = -1
    total_sum = 0
    total_count = 0

    for line in fTest:
        userID, trackID, albumID, artistID, *_ = line.strip().split('|')


        if userID != lastUserID:
            ii = 0
            user_rating_inTrain = np.zeros(shape=(6, 3))

        trackID_vec[ii] = trackID
        albumID_vec[ii] = albumID
        artistID_vec[ii] = artistID
        ii += 1
        lastUserID = userID

        if ii == 6:
            user_train_data = train_data[userID]

            for nn in range(6):
                user_rating_inTrain[nn] = [user_train_data.get(trackID_vec[nn], 0),
                                           user_train_data.get(
                                               albumID_vec[nn], 0),
                                           user_train_data.get(artistID_vec[nn], 0)]

            for nn in range(6):
                total_score = sum(user_rating_inTrain[nn])
                total_sum += total_score
                total_count += 1
                prediction = int(total_sum / total_count > 58)

                csv_writer.writerow([userID, trackID_vec[nn], albumID_vec[nn],
                                    artistID_vec[nn], *user_rating_inTrain[nn], total_score, prediction])

print("Total:" + str(total_sum))
print("Count: " + str(total_count))
print("Average: " + str(total_sum/total_count))


Total:7044150.0
Count: 120000
Average: 58.70125
